In [ ]:
%run init_notebook.py

import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio.transforms as T
import time
import torchaudio
from tqdm import tqdm
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from IPython.display import Audio, display

from src.dataset import NSynth   
from src.models import AutoEncoder
from src.utils.dataset import load_raw_waveform
from src.utils.logger import save_training_results
from src.utils.models import compute_magnitude_and_phase

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = r"C:\Users\Articuno\Desktop\TFG-info\data\models\autoencoder.pth"

# STFT transform
sample_rate = 16000
n_fft = 4000 # Small n_fft: good time resolution, poor frequency resolution. Large n_fft: good frequency resolution, poor time resolution.
hop_length = 300
win_length = n_fft  # Same as n_fft

# Apply the correct transform for magnitude and phase (onesided=False to handle complex spectrogram)
stft_transform = T.Spectrogram(
    n_fft=n_fft,
    win_length=win_length,
    hop_length=hop_length,
    power=None,  # Keep as complex spectrogram (magnitude and phase)
    onesided=False,  # Make sure we keep the full spectrum (complex-valued)
    center=False
).to(device)

# Define the inverse STFT function (onesided=False)
istft_transform = torchaudio.transforms.InverseSpectrogram(
    n_fft=n_fft,
    win_length=win_length,
    hop_length=hop_length,
    onesided=False  # Make sure we reconstruct using the full complex spectrogram
).to(device)

# Datasets and DataLoaders, training parameters.
train_dataset = NSynth(partition='training')
valid_dataset = NSynth(partition='validation')
test_dataset  = NSynth(partition='testing')

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

input_height = 4000 # = n_fft because onesided=False
input_width  = 201 # = 1 + floor((T - n_ftt) / hop_length), where T is the whole duration (4 * 16000 = 64000)
latent_dim   = 200
learning_rate = 1e-4

model = AutoEncoder((input_height, input_width), latent_dim).to(device)

if os.path.exists(model_path):
    try:
        model.load_state_dict(torch.load(model_path))
        print("Model loaded successfully.")
    except PermissionError as e:
        print(f"PermissionError: {e}. Unable to load the model.")
else:
    print("No saved model found, starting training from scratch.")

    # Model, Optimizer, and Loss
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss().to(device)

    # Training Loop
    num_epochs = 20
    log_interval = 10
    avg_epoch_time = 0.0

    training_losses = []
    validation_losses = []

    # Learning rate scheduler to reduce learning rate based on validation loss
    scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)

    print(f"Starting training on {device}...")
    for epoch in tqdm(range(num_epochs), desc="Training"):
        model.train()
        start_epoch_time = time.time()
        train_loss = 0.0
        
        # Training Loop
        for i, (waveform, _, _, _) in enumerate(train_loader):
            # waveform has shape [batch_size, 1, time_steps]

            waveform = waveform.to(device)

            # Apply STFT transformation
            stft_spec = stft_transform(waveform)  # [batch_size, 1, freq_bins, time_frames]

            # Extract magnitude and phase
            magnitude, phase = compute_magnitude_and_phase(stft_spec)

            input_data = torch.cat([magnitude, phase], dim=1).to(device)  # [batch_size, 2, freq_bins, time_frames]

            optimizer.zero_grad()
            output = model(input_data)  # shape [batch_size, 2, freq_bins, time_frames]

            loss = criterion(output, input_data)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * waveform.size(0)

        # Compute average epoch loss for training
        train_loss /= len(train_loader.dataset)

        # Validation Loop
        model.eval()
        valid_loss = 0.0
        with torch.no_grad():
            for i, (waveform, _, _, _) in enumerate(valid_loader):
                waveform = waveform.to(device)
                stft_spec = stft_transform(waveform)
                
                # Extract magnitude and phase
                magnitude, phase = compute_magnitude_and_phase(stft_spec)
                input_data = torch.cat([magnitude, phase], dim=1).to(device)

                output = model(input_data)
                loss = criterion(output, input_data)
                valid_loss += loss.item() * waveform.size(0)

        # Compute average validation loss
        valid_loss /= len(valid_loader.dataset)
        
        # Step the scheduler with validation loss
        scheduler.step(valid_loss)

        epoch_time = time.time() - start_epoch_time
        avg_epoch_time += epoch_time
        training_losses.append(train_loss)
        validation_losses.append(valid_loss)

        print(f"Epoch {epoch+1}, train_loss={train_loss}, valid_loss={valid_loss}, Time: {epoch_time:.2f}s")
            
    print("Training complete. Saving results...")

    save_training_results({
        "train_loss": training_losses,
        "valid_loss": validation_losses,
        "num_epochs": num_epochs,
        "avg_epoch_time": avg_epoch_time / num_epochs,
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "sample_rate": sample_rate,
        "n_fft": n_fft,
        "hop_length": hop_length,
        "input_height": input_height,
        "input_width": input_width,
        "latent_dim": latent_dim,
    }, "autoencoder.json")

    torch.save(model.state_dict(), model_path)
    # Plot training and validation losses
    import matplotlib.pyplot as plt
    epochs = range(1, num_epochs + 1)

    plt.figure(figsize=(8, 5))
    plt.plot(epochs, training_losses, label='Training Loss', marker='o')
    plt.plot(epochs, validation_losses, label='Validation Loss', marker='x')
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training vs Validation Loss")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()




Encoder:  Encoder(
  (encoder): Sequential(
    (0): Sequential(
      (0): Conv2d(2, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): ReLU()
      (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Sequential(
      (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): ReLU()
      (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): ReLU()
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=832000, out_features=200, bias=True)
  (fc2): Linear(in_features=832000, out_features=200, bias=True)
)
Decoder:  Decoder(
  (fc): Linear(in_features=200, out_features=832000, bias=True)
  (unflatten): Unflatten(dim=1, unflattened_size=(64,

Training:   0%|          | 0/20 [00:00<?, ?it/s]

### Reconstruction Accuracy

The **reconstruction accuracy** for each sample is computed as:

$$
\text{Accuracy}_i = \max \left(0, 1 - \frac{\lVert x_i - \hat{x}_i \rVert_2^2}{\lVert x_i \rVert_2^2 + \varepsilon} \right) \times 100\%
$$

where:
- $x_i$ is the input for sample $i$ (of shape `[batch_size, 2, num_freq, time_frames]`),
- $\hat{x}_i$ is the reconstructed output from the model,
- $\lVert \cdot \rVert_2^2$ denotes the squared L2 norm (i.e., the sum of squared elements),
- $\varepsilon$ is a small constant to avoid division by zero.

This gives a value between 0% and 100% for each sample.

---

### Total Accuracy

To compute the **overall accuracy** across all samples in the dataset (or batch-wise using a DataLoader), we take the average of all per-sample accuracies:

$$
\text{Total Accuracy} = \frac{1}{N} \sum_{i=1}^{N} \text{Accuracy}_i
$$

where $N$ is the total number of samples evaluated.

This metric tells us how well the model reconstructs the inputs **on average** over the dataset.


In [ ]:
def reconstruction_accuracy(x, x_hat):
    # Compute per-sample accuracy tensor of shape [B]
    numerator = torch.sum((x - x_hat) ** 2, dim=(1, 2, 3))
    denominator = torch.sum(x ** 2, dim=(1, 2, 3)) + 1e-8
    accuracy = 1 - numerator / denominator
    return torch.clamp(accuracy, min=0.0, max=1.0)


model.eval()
total_acc_sum = 0.0
total_samples = 0

with torch.no_grad():
    for (waveform, _, _, _) in test_loader:
        raw_waveform = waveform.to(device)  # shape: [B, 1, T]

        stft_spec = stft_transform(raw_waveform)
        magnitude, phase = compute_magnitude_and_phase(stft_spec)
        input = torch.cat([magnitude, phase], dim=1)

        output = model(input)

        batch_acc = reconstruction_accuracy(input, output)  # shape: [B]
        total_acc_sum += batch_acc.sum().item()
        total_samples += waveform.size(0)

test_accuracy = round((total_acc_sum / total_samples) * 100, 2)
print(f"Testing accuracy: {test_accuracy}%")

Testing accuracy: 71.18%


In [ ]:
# --------------------------
# Audio Playback
# --------------------------

from src.config import MAX_MAGNITUDE, MAX_PHASE

model.eval()

# Samples to compare
test_indices = [random.choice(range(len(test_dataset))) for _ in range(10)]

for idx in test_indices:
    print(f"\n=== Test sample index: {idx} ===")
    # (stft_spec, sample_rate, key, metadata) from dataset; here we only need key and sample_rate
    _, sample_rate, key, metadata = test_dataset[idx]

    # Listen to the Original Audio with no transform applied
    raw_waveform, raw_sr = load_raw_waveform("testing", key)
    print(f"Key: {key}")
    print("Original audio:")
    display(Audio(raw_waveform.numpy(), rate=raw_sr))

    # Ensure raw_waveform has a batch and channel dimension, expected shape: [batch, 1, time_steps]
    if raw_waveform.ndim == 2:  # e.g., [1, time_steps]
        raw_waveform = raw_waveform.unsqueeze(0)
    
    raw_waveform = raw_waveform.to(device)

    # Apply STFT transformation
    stft_spec = stft_transform(raw_waveform)  # [batch, 1, freq_bins, time_frames]
    
    # Extract magnitude and phase using the helper function
    magnitude, phase = compute_magnitude_and_phase(stft_spec)  # both: [batch, 1, freq_bins, time_frames]
    input_data = torch.cat([magnitude, phase], dim=1).to(device)
    
    # Reconstruct using the model
    with torch.no_grad():
        output_data = model(input_data)  # shape: [batch, 2, freq_bins, time_frames]
    
    # Split the network output into magnitude and phase
    recon_magnitude = output_data[:, 0, :, :]  # [batch, freq_bins, time_frames]
    recon_phase = output_data[:, 1, :, :]      # [batch, freq_bins, time_frames]

    # Recombine into a complex spectrogram
    recon_complex = recon_magnitude * torch.exp(1j * recon_phase)
    
    # Apply the inverse STFT transformation to get the waveform
    reconstructed_waveform = istft_transform(recon_complex)  # [batch, time_steps]
    
    print("Reconstructed audio:")
    display(Audio(reconstructed_waveform.squeeze().cpu().numpy(), rate=raw_sr))


=== Test sample index: 3197 ===
Key: guitar_electronic_022-072-025
Original audio:


Reconstructed audio:



=== Test sample index: 2982 ===
Key: guitar_electronic_022-046-025
Original audio:


Reconstructed audio:



=== Test sample index: 972 ===
Key: flute_acoustic_002-067-050
Original audio:


...

Reconstructed audio:



=== Test sample index: 1892 ===
Key: vocal_synthetic_003-060-100
Original audio:


Reconstructed audio:



=== Test sample index: 3476 ===
Key: guitar_acoustic_015-051-025
Original audio:


Reconstructed audio:
